In [1]:
from skmap.io import RasterData
from skmap.io import process
import numpy as np
from skmap.misc import ttprint
import pandas as pd
import traceback
import sys
from rasterio.windows import Window
from typing import Callable, Iterator, List,  Union
from osgeo import gdal, gdal_array
import numpy as np
from datetime import datetime
import skmap_bindings
import pandas as pd
import numpy as np
from pathlib import Path
import random

gdal_opts = {
 'GDAL_HTTP_VERSION': '1.0',
 'CPL_VSIL_CURL_ALLOWED_EXTENSIONS': '.tif',
}
n_threads = 96

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
df = pd.read_csv("/mnt/nibble/VITO_GDP/new_list.in", header=None)
df = df.rename(columns={0: 'file'})
df['date'] = df['file'].apply(lambda f: f.split('_')[3][0:8])
# df['file'] = df['file'].apply(lambda f: f"http://192.168.49.30:8333/tmp-global-geomorpho/GDP/{f}" )
df['file'] = df['file'].apply(lambda f: f"../data/{f}" )
df = df.sort_values('date')
df['dt1'] = pd.to_datetime(df['date'], format='%Y%m%d')
df['dt2'] = np.roll(df['dt1'],-1)
df['dt2'].iloc[df.shape[0]-1] = pd.to_datetime('20231231', format='%Y%m%d')

df['file'] = df['file'].apply(lambda x: x.replace('192.168.49.30', f'192.168.49.{30+random.randint(0,13)}'))

files_list = df['file'].tolist()[200:208]
files_list[0:3]

/tmp/ipykernel_2592/2902411329.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dt2'].iloc[df.shape[0]-1] = pd.to_datetime('20231231', format='%Y%m%d')


['../data/c_gls_GDMP300-RT5_201907310000_GLOBE_PROBAV_V1.0.1_gdp.tiff',
 '../data/c_gls_GDMP300-RT5_201908100000_GLOBE_PROBAV_V1.0.1_gdp.tiff',
 '../data/c_gls_GDMP300-RT5_201908200000_GLOBE_PROBAV_V1.0.1_gdp.tiff']

In [3]:
percentiles = [25., 50., 75.]
bands_list = [1,]
n_rasters = len(files_list)
file_order = np.arange(n_rasters)
x_off = 0
y_off = 0
# x_size, y_size = (100, 100)
x_size, y_size = (120960, 47040)
shape = (n_rasters, x_size * y_size)
array = np.empty(shape, dtype=np.float32)
array_t = np.empty((shape[1], shape[0]), dtype=np.float32)
array_perc_t = np.empty((shape[1], len(percentiles)), dtype=np.float32)

In [4]:
%%time
skmap_bindings.readData(array, n_threads, files_list, file_order, x_off, y_off, x_size, y_size, bands_list, gdal_opts)

CPU times: user 30.7 ms, sys: 18.6 ms, total: 49.3 ms
Wall time: 7.6 ms


In [5]:
%%time
skmap_bindings.transposeArray(array, n_threads, array_t)

CPU times: user 166 µs, sys: 26 µs, total: 192 µs
Wall time: 67.7 µs


In [6]:
%%time
skmap_bindings.computePercentiles(array_t, 3, array_perc_t, percentiles)

CPU times: user 4.4 ms, sys: 0 ns, total: 4.4 ms
Wall time: 580 µs


In [7]:
print(array_t[0:10,:])
print(array_perc_t[0:10,:])

[[23. 15. 23. 33. 27. 34. 58. 84.]
 [23. 15. 23. 33. 27. 34. 58. 84.]
 [23. 15. 23. 22. 18. 34. 59. 84.]
 [23.  7. 12. 22. 18. 23. 44. 63.]
 [12.  7. 12. 11.  9. 23. 29. 63.]
 [12.  7. 12. 22. 18. 23. 44. 63.]
 [23.  7. 12. 22. 18. 23. 29. 63.]
 [12.  7. 12. 22. 18. 23. 29. 63.]
 [12.  7. 12. 22. 18. 23. 29. 42.]
 [12.  8. 12. 22.  9. 12. 30. 63.]]
[[23.   30.   40.  ]
 [23.   30.   40.  ]
 [21.   23.   40.25]
 [16.5  22.5  28.25]
 [10.5  12.   24.5 ]
 [12.   20.   28.25]
 [16.5  22.5  24.5 ]
 [12.   20.   24.5 ]
 [12.   20.   24.5 ]
 [11.25 12.   24.  ]]
